In [43]:
from random import shuffle
import glob,h5py, cv2, scipy, pickle, itertools, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm
subj = 'caroline'
filename = 'Datasets/'+'final_'+subj+'_spec2_128_last.pkl' # no resize
print('File to be created: ' + filename)

File to be created: Datasets/final_caroline_spec2_128_last.pkl


In [44]:
# load labels
# subjects = ['10 dec emre mahbub', '12 jan emre', '11 jan mahbub emre ladi', '12 jan ozgur mahbub ladi emre', 
#             '13 jan ridvan ademola sean akthar emre', '14 jan ashwanth zeki', '15 apr ozgur',
#            '15 jan emin', '19 apr sean', '19 apr sevgi', '21 apr emin', '22 apr ladi', '22 apr sean',
#            '23 apr alperen', '23 apr mahbub', '5 apr sevgi']
# subjects = ['6 apr blake', '15 apr darrin', '16 apr devon asl2', '20 apr caroline', '21 apr blake']
subjects = ['20 apr caroline']

mainpath = '/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/'
label_files = []
for i in range(len(subjects)):
    addr = glob.glob(mainpath+subjects[i]+'/labels/microDoppler/'+'*.txt')
    label_files.append(addr)
label_files = sum(label_files, [])
print('Num. of label files: '+str(len(label_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+label_files[i])

Num. of label files: 125
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/labels/microDoppler/11010001_1618952365_2.txt
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/labels/microDoppler/11010001_1618951328_3.txt
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/labels/microDoppler/11020001_1618950528_1.txt
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/labels/microDoppler/11050001_1618953792_1.txt
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/labels/microDoppler/11040001_1618952922_4.txt


In [45]:
# load videos
im_files = []
for i in range(len(label_files)):
    addr = label_files[i].replace('labels/','').replace('txt','png')
    im_files.append(addr)
# RD_files = sum(RD_files, [])
print('Num. of label files: '+str(len(im_files)))
for i in range(0,5):    
    print(str(i+1)+'. '+im_files[i])

Num. of label files: 125
1. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/microDoppler/11010001_1618952365_2.png
2. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/microDoppler/11010001_1618951328_3.png
3. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/microDoppler/11020001_1618950528_1.png
4. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/microDoppler/11050001_1618953792_1.png
5. /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/microDoppler/11040001_1618952922_4.png


In [46]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return np.array(dataframe.values)

In [47]:
envelopes = []
envs = glob.glob('/mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/*/envelopes/*.txt')
print(len(envs))
for i in tqdm(range(len(im_files)), position = 0, leave = True):
    fname = im_files[i].replace('png','txt').replace('microDoppler','envelopes')
#     idx = [x for x in range(len(envs)) if re.search(fname, envs[x])]
#     assert len(idx) == 1
    envelopes.append(np.squeeze(load_file(fname)))
envelopes = np.array(envelopes) 

  1%|          | 1/125 [00:00<00:14,  8.52it/s]

1970


100%|██████████| 125/125 [00:11<00:00, 10.83it/s]


In [48]:
def pad_labels(yx):
    numpad = duration*fps - yx.shape[1]
    pads = np.repeat(yx[0,-1],numpad)
    padded = np.expand_dims(np.append(yx,pads),0)
    return padded

In [49]:
dur = 24 # sec
win = 0.2 # win len
n_step = int(dur/win)
des_w = 128*n_step
des_h = 128
fps = 25
print('num steps: '+str(n_step))
print('desired width: '+str(des_w))

num steps: 120
desired width: 15360


In [50]:
labels = []

cnt = 1
for file in label_files:
    clear_output(wait=True)
    print('Loading training labels #'+str(cnt))
    yx = load_file(file)
#     if yx.shape[1] < dur*fps:
#         yx = pad_labels(yx)
    idx = np.linspace(0,yx.shape[1]-1,n_step).astype(int)
    labels.append(yx[0,idx])
#     labels.append(yx)
    cnt += 1
    
labels = np.array(labels)    
# labels = to_categorical(np.swapaxes(labels,1,2))
labels = to_categorical(labels)
idx = np.linspace(0,labels.shape[1]-1,n_step).astype(int)
labels = labels[:,idx,:]
print(labels.shape)

Loading training labels #125
(125, 120, 19)


In [51]:
idx = np.linspace(0,labels.shape[1]-1,n_step).astype(int)
labels = labels[:,idx,:]
print(labels.shape)

(125, 120, 19)


In [52]:
train_spec, test_spec, y_train, y_test, env_train, env_test = train_test_split(im_files, labels, envelopes, test_size=0.2, random_state=1)

In [53]:
def resizer(im, des_w, des_h, n_step):
    resized = np.reshape(cv2.resize(im, (des_w, des_h), interpolation = cv2.INTER_AREA),(n_step, des_h, 128, 3))    
    return resized

In [54]:
cnt = 1
train_images = []

for image in train_spec:
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    img = resizer(img, des_w, des_h, n_step)
    clear_output(wait=True)
    print('Loading training image '+str(cnt)+'/'+str(len(train_spec))+ ' '+image)
    print('orig shape: '+str(img.shape))
    train_images.append(img/255.)  
    cnt += 1
train_images = np.array(train_images)
cnt = 1
test_images = []

for image in test_spec:
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    img = resizer(img, des_w, des_h, n_step)
    clear_output(wait=True)
    print('Loading test image '+str(cnt)+'/'+str(len(test_spec))+ ' '+image)
    print('orig shape: '+str(img.shape))
    test_images.append(img/255.)  
    cnt += 1
test_images = np.array(test_images)

print(train_images.shape)
print(test_images.shape)

Loading test image 25/25 /mnt/HDD01/rspl-admin/DATASETS/Fall Sequential/Outputs/20 apr caroline/microDoppler/11020001_1618951472_1.png
orig shape: (120, 128, 128, 3)
(100, 120, 128, 128, 3)
(25, 120, 128, 128, 3)


In [55]:
data = [train_images[0:800], test_images[0:200], y_train[0:800], y_test[0:200], env_train[0:800], env_test[0:200]]
with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
print(filename+' created.')

Datasets/final_caroline_spec2_128_last.pkl created.


In [56]:
# import gc
# gc.collect()